In [ ]:
import os
import torch
import numpy as np
import cv2
from PIL import Image
from modeling.deeplab import DeepLab

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import argparse

In [ ]:
parser = argparse.ArgumentParser(description="PyTorch DeeplabV3Plus Inference")
parser.add_argument('--backbone', type=str, default='mobilenet',
                    choices=['resnet', 'xception', 'drn', 'mobilenet'],
                    help='backbone name (default: resnet)')
parser.add_argument('--out-stride', type=int, default=16,
                    help='network output stride (default: 8)')
parser.add_argument('--nclass', type=int, default=21,
                    help='number of classes(incluce background class)')
parser.add_argument('--checkpoint', type=str, default=r"C:\Users\CUONG\Desktop\model_best.pth.tar",
                    help='Checkpoint file path')
parser.add_argument('--size', type=str, default='(513, 513)',
                    help='Input size')
args = parser.parse_args('')

In [ ]:
args

In [ ]:
def load_model(args):
    model = DeepLab(num_classes=args.nclass,
                    backbone=args.backbone,
                    output_stride=args.out_stride)
    checkpoint = torch.load(args.checkpoint, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    return model

In [ ]:
model = load_model(args)

In [ ]:
def inference_file(model, args, image_path):
    img = cv2.imread(image_path)
    img = Image.fromarray(img)
    img = img.resize(eval(args.size), Image.BILINEAR)
    img = np.array(img).astype(np.float32)
    img /= 255.0
    img -= (0.485, 0.456, 0.406)
    img /= (0.229, 0.224, 0.225)
    img = np.array(img).astype(np.float32).transpose((2, 0, 1))
    img = torch.from_numpy(img).float()
    img = img.unsqueeze(0)
    output = model(img)
    output = output[0]
    predict = torch.argmax(output, dim=0)
    predict = predict.cpu().numpy()
    return predict

In [ ]:
predict = inference_file(model, args, r"E:\JupyterNotebook\dataset\hair\CelebAMask-HQ\CelebA-HQ-img\13961.jpg")

In [ ]:
predict[predict > 0] = 255
predict = np.stack([np.zeros_like(predict), np.zeros_like(predict), predict], axis=-1)
predict = predict.astype(np.uint8)

In [ ]:
plt.imshow(predict)

In [ ]:
image = cv2.imread(r"E:\JupyterNotebook\dataset\hair\CelebAMask-HQ\CelebA-HQ-img\13961.jpg")

In [ ]:
image = cv2.resize(image, predict.shape[: 2], cv2.INTER_LINEAR)

In [ ]:
blend = cv2.addWeighted(image, 0.8, predict, 0.2, 0.0)

In [ ]:
plt.imshow(blend)

In [ ]:
cv2.imwrite(r"E:\JupyterNotebook\dataset\hair\test.jpg", blend)

In [1]:
!python inference.py --dir_path "E:\JupyterNotebook\dataset\hair\CelebAMask-HQ\CelebA-HQ-img" \
                     --output_dir "E:\JupyterNotebook\dataset\hair\CelebAMask-HQ\output" \
                     --max_file 10

Predicting:  E:\JupyterNotebook\dataset\hair\CelebAMask-HQ\CelebA-HQ-img\0.jpg
Predicting:  E:\JupyterNotebook\dataset\hair\CelebAMask-HQ\CelebA-HQ-img\1.jpg
Predicting:  E:\JupyterNotebook\dataset\hair\CelebAMask-HQ\CelebA-HQ-img\10.jpg
Predicting:  E:\JupyterNotebook\dataset\hair\CelebAMask-HQ\CelebA-HQ-img\100.jpg
Predicting:  E:\JupyterNotebook\dataset\hair\CelebAMask-HQ\CelebA-HQ-img\1000.jpg
Predicting:  E:\JupyterNotebook\dataset\hair\CelebAMask-HQ\CelebA-HQ-img\10000.jpg
Predicting:  E:\JupyterNotebook\dataset\hair\CelebAMask-HQ\CelebA-HQ-img\10001.jpg
Predicting:  E:\JupyterNotebook\dataset\hair\CelebAMask-HQ\CelebA-HQ-img\10002.jpg
Predicting:  E:\JupyterNotebook\dataset\hair\CelebAMask-HQ\CelebA-HQ-img\10003.jpg
Predicting:  E:\JupyterNotebook\dataset\hair\CelebAMask-HQ\CelebA-HQ-img\10004.jpg


OMP: Warning #96: Cannot form a team with 2 threads, using 1 instead.
OMP: Hint Consider unsetting KMP_DEVICE_THREAD_LIMIT (KMP_ALL_THREADS), KMP_TEAMS_THREAD_LIMIT, and OMP_THREAD_LIMIT (if any are set).
